In [1]:
# Decision Tree Implementation From Scratch
# GSB Training 2025 - Class 2 Assignment

import numpy as np
from collections import Counter

Prediksi: ['No', 'Yes', 'Yes']


In [2]:
# 1. Hitung impurity (Gini)
def gini(y):
    counts = Counter(y)
    impurity = 1
    for label in counts:
        prob = counts[label] / len(y)
        impurity -= prob ** 2
    return impurity

In [3]:
# 2. Split dataset
def split_dataset(X, y, feature, value):
    left_idx = [i for i in range(len(X)) if X[i][feature] <= value]
    right_idx = [i for i in range(len(X)) if X[i][feature] > value]

    X_left = [X[i] for i in left_idx]
    y_left = [y[i] for i in left_idx]
    X_right = [X[i] for i in right_idx]
    y_right = [y[i] for i in right_idx]

    return X_left, y_left, X_right, y_right

In [4]:
# 3. Cari split terbaik
def best_split(X, y):
    best_gain = 0
    best_feature, best_value = None, None
    current_impurity = gini(y)

    n_features = len(X[0])
    for feature in range(n_features):
        values = set([row[feature] for row in X])
        for val in values:
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature, val)

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            # weighted gini
            p = len(y_left) / len(y)
            impurity = p * gini(y_left) + (1 - p) * gini(y_right)
            gain = current_impurity - impurity

            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_value = val

    return best_feature, best_value, best_gain


In [5]:
# 4. Node Class
class Node:
    def __init__(self, feature=None, value=None, left=None, right=None, *, label=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.label = label

In [6]:
# 5. Build tree
def build_tree(X, y, depth=0, max_depth=5):
    # stop condition
    if len(set(y)) == 1:
        return Node(label=y[0])
    if depth >= max_depth:
        return Node(label=Counter(y).most_common(1)[0][0])

    feature, value, gain = best_split(X, y)
    if gain == 0:
        return Node(label=Counter(y).most_common(1)[0][0])

    X_left, y_left, X_right, y_right = split_dataset(X, y, feature, value)
    left_branch = build_tree(X_left, y_left, depth+1, max_depth)
    right_branch = build_tree(X_right, y_right, depth+1, max_depth)

    return Node(feature, value, left_branch, right_branch)


In [7]:
# 6. Prediksi
def predict_one(node, sample):
    if node.label is not None:
        return node.label
    if sample[node.feature] <= node.value:
        return predict_one(node.left, sample)
    else:
        return predict_one(node.right, sample)

def predict(tree, X):
    return [predict_one(tree, sample) for sample in X]

In [8]:
# 7. Contoh penggunaan
if __name__ == "__main__":
    # Dataset sederhana: [Outlook, Temperature], Label = Main Tennis
    # 0 = cerah, 1 = mendung, 2 = hujan
    # Temperature = angka (derajat)
    X = [
        [0, 30],
        [0, 25],
        [1, 28],
        [2, 20],
        [2, 23],
        [1, 21],
        [0, 24],
    ]
    y = ["No", "No", "Yes", "Yes", "Yes", "Yes", "No"]

    tree = build_tree(X, y, max_depth=3)

    test_data = [[0, 26], [2, 22], [1, 27]]
    print("Prediksi:", predict(tree, test_data))

Prediksi: ['No', 'Yes', 'Yes']
